In [34]:
import sympy as sp
import inspect
import re

In [35]:
def python_to_matlab(python_code, new_name):
    """
    Converts Python code to equivalent MATLAB code.

    Args:
        python_code (str): The Python code to be translated.
        new_name (str): The new function name for the MATLAB code.

    Returns:
        str: The translated MATLAB code.
    """

    # Replace mathematical operators
    matlab_code = re.sub(r'\*\*', r'.^', python_code)  # ** -> .^
    matlab_code = re.sub(r'\*', r'.*', matlab_code)    # * -> .*
    matlab_code = re.sub(r'/', r'./', matlab_code)     # / -> ./

    # Add semicolons
    matlab_code = re.sub(r'(?m)^.*$', r'\g<0>;', matlab_code)  # Add semicolon at the end of each line

    # Replace array syntax
    pattern = r"array\(\[(\[.*?\])\]\)"

    def repl(match):
        inner_content = match.group(1).replace('], [', ', ...\n                  ')
        formatted_content = inner_content.replace('[', '').replace(']', '')
        return f"result = [{formatted_content}];"
    matlab_code = re.sub(pattern, repl, matlab_code)

    # Detect and replace function definition
    def repl_func_def(match):
        args = match.group(0).replace('def _lambdifygenerated(', '').replace('):', '')
        args_list = args.split(', ')
        if len(args_list) <= 6:
            return f"function result = {new_name}(input_data)\n" \
                   "    X = input_data(:, 1);\n" \
                   "    Y = input_data(:, 2);\n" \
                   "    Z = input_data(:, 3);\n" \
                   "    alpha = input_data(:, 4);\n" \
                   "    beta = input_data(:, 5);\n" \
                   "    gama = input_data(:, 6);"
        else:
            obs_vars = "\n".join([f"    f_obs{i} = f_obs({i});" for i in range(1, len(args_list) - 5)])
            return f"function result = {new_name}(input_data, f_obs)\n" \
                   "    X = input_data(:, 1);\n" \
                   "    Y = input_data(:, 2);\n" \
                   "    Z = input_data(:, 3);\n" \
                   "    alpha = input_data(:, 4);\n" \
                   "    beta = input_data(:, 5);\n" \
                   "    gama = input_data(:, 6);\n" \
                   f"{obs_vars}"

    # Apply function definition replacement
    matlab_code = re.sub(r"def _lambdifygenerated\([^\)]+\):", repl_func_def, matlab_code, 1)
    matlab_code = matlab_code.replace(';;', ';')

    # Remove last line if it starts with ;
    if matlab_code.endswith(';'):
        matlab_code = matlab_code[:-1]

    matlab_code = matlab_code.replace('return', 'result =')
    matlab_code = matlab_code.replace('result = result', 'result')
    
    return matlab_code

# Kamera operator positions and angles -> 5 point calibration projection into 2D plane

- input X, Y, Z in meters
- input alpha, beta, gamma in radians
- output x, y in milimeters

In [36]:
# input parameters

# positions of calibration points in 3D
M = sp.Matrix(5, 3, (0, 0, 0, 0.05, 0.05, 0, 0.05, -0.05, 0, -0.05, -0.05, 0, -0.05, 0.05, 0))

# focal length
ff = 0.008

# length of the robot arm + length of the end effector
L = 309.5 * 1e-3 

# Define the vectors v and n
v = sp.Matrix([0, 0, -1])
n = sp.Matrix([0, 1, 0])


##########################################
# Camera projection to the plane         #
##########################################

# Define the symbols
alpha, beta, gama, X, Y, Z, S_x, S_y, S_z = sp.symbols(
    'alpha beta gama X Y Z S_x S_y S_z', real=True)

# Sip and TCP
Sip = sp.Matrix([S_x, S_y, S_z])
TCP = sp.Matrix([X, Y, Z])

# Rotation matrices Rx, Ry, Rz
Rx = sp.Matrix([[1, 0, 0],
                [0, sp.cos(alpha), -sp.sin(alpha)],
                [0, sp.sin(alpha), sp.cos(alpha)]])
Ry = sp.Matrix([[sp.cos(beta), 0, sp.sin(beta)],
                [0, 1, 0],
                [-sp.sin(beta), 0, sp.cos(beta)]])
Rz = sp.Matrix([[sp.cos(gama), -sp.sin(gama), 0],
                [sp.sin(gama), sp.cos(gama), 0],
                [0, 0, 1]])
R = sp.simplify(Rx * Ry * Rz)


# Vector v and calculations for x and o
x = sp.Matrix([X, Y, Z]) - L * R * v
o = R * v

# Calculate 'a' using dot products
a = sp.sqrt(((-R.row(0).dot(v) * Z) / (R.row(2).dot(v)))**2 + 
            (-(R.row(1).dot(v) * Z) / (R.row(2).dot(v)))**2 + Z * Z)

# Calculate Q
Q = a * o + TCP

# Project Sip onto the line defined by o and Q
Sip_proj = Sip + (o.dot(Q) - o.dot(Sip)) * (x - Sip) / (o.dot(x) - o.dot(Sip))

# s_2 vector and s_1 as the cross product of o and s_2
s_2 = R * n
s_1 = o.cross(s_2)

# Calculate k_1 and k_2
k_1 = (Sip_proj - Q).dot(s_1) / s_1.norm()**2
k_2 = (Sip_proj - Q).dot(s_2) / s_2.norm()**2


# Calculate image plane coordinates f_1 and f_2
f_1 = k_1 * ff / (a + L - ff)
f_2 = k_2 * ff / (a + L - ff)


##########################################
# Transformation for all points in M     #
##########################################

# Define the matrix to store the results
results = sp.Matrix(10, 1, [0] * 10)

# Define the loop
for i in range(5):
    # Substitute the symbols from M for S_x, S_y, S_z in f_1 and f_2
    f1_i = f_1.subs({S_x: M[i, 0], S_y: M[i, 1], S_z: M[i, 2]})
    f2_i = f_2.subs({S_x: M[i, 0], S_y: M[i, 1], S_z: M[i, 2]})

    # Store the results in the matrix
    results[2 * i, 0] = f1_i * 1e3
    results[2 * i + 1, 0] = f2_i * 1e3

## Kamera operator export

In [37]:
v = sp.Matrix([X, Y, Z, alpha, beta, gama])
kamera_op = sp.lambdify(v, results, modules='numpy', cse=True)

with open('kamera.m', 'w') as f:
    file_text = python_to_matlab(inspect.getsource(kamera_op), 'kamera')
    f.write(file_text)
    

# F_val operator

In [38]:

f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10 = sp.symbols('f_obs1:11', real=True)

# Define the symbols
f_obs = sp.Matrix(10, 1, (f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10))
diff = results - f_obs
F_obs = (diff[0] * diff[0] + diff[1] * diff[1] +
         diff[2] * diff[2] + diff[3] * diff[3] +
         diff[4] * diff[4] + diff[5] * diff[5] +
         diff[6] * diff[6] + diff[7] * diff[7] +
         diff[8] * diff[8] + diff[9] * diff[9])


## F_val operator export

In [39]:
v = sp.Matrix([X, Y, Z, alpha, beta, gama, f_obs1, f_obs2, f_obs3,
              f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10])

F_obs_op = sp.lambdify(v, F_obs, modules='numpy', cse=True)

with open('F_val.m', 'w') as f:
    file_text = python_to_matlab(inspect.getsource(F_obs_op), 'F_val')
    f.write(file_text)

# Gradient of F_val

In [40]:
args = sp.Matrix([X, Y, Z, alpha, beta, gama])
grad_F = sp.derive_by_array(F_obs, args)

## Gradient of F_val export

In [41]:
v = sp.Matrix([X, Y, Z, alpha, beta, gama, f_obs1, f_obs2, f_obs3,
              f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10])
grad_F_op = sp.lambdify(v, grad_F, modules='numpy', cse=True)

with open('F_grad.m', 'w') as f:
    file_text = python_to_matlab(inspect.getsource(grad_F_op), 'F_grad')
    f.write(file_text)

# Hessians of F_val

In [42]:
args = sp.Matrix([X, Y, Z, alpha, beta, gama])
hess_F = sp.hessian(F_obs, args)

## Hessians of F_val export

In [43]:
v = sp.Matrix([X, Y, Z, alpha, beta, gama, f_obs1, f_obs2, f_obs3,
              f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10])
hess_F_op = sp.lambdify(v, hess_F, modules='numpy', cse=True)

with open('F_hess.m', 'w') as f:
    file_text = python_to_matlab(inspect.getsource(hess_F_op), 'F_hess')
    f.write(file_text)